Practice problems
=================



This directory is designed to be "org-published". The idea is to have org files that setup practice problems with links to solutions.



In [1]:
(defun ox-ipynb-publish-practice (plist filename pub-dir)
  "Publish an org-file to a Jupyter notebook."
  (let ((buf-open (find-buffer-visiting filename))
	(buf (find-file-noselect filename))
	(ipynb (file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))

    (with-current-buffer buf
      (let* ((ox-ipynb-preprocess-hook
	      (append ox-ipynb-preprocess-hook
		      `((lambda ()
			  (goto-char (point-min))
			  (insert (format "[[https://colab.research.google.com/github/jkitchin/f21-06623/blob/master/practice/notebooks/%s][Open in colab]]\n\n"
					  ,(file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))
			  (goto-char (point-max))

			  (insert (format "\n\n* Solutions\n\n [[https://colab.research.google.com/github/jkitchin/f21-06623/blob/master/practice/solutions/%s][Open in colab]]\n\n"
					  ,(file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb")))))
			ox-ipynb-remove-hidden
			ox-ipynb-remove-solution
			ox-ipynb-remove-remove)))
	     (output (ox-ipynb-export-to-ipynb-file)))
	(org-publish-attachment plist (expand-file-name output) pub-dir)
	(unless buf-open (kill-buffer buf))
	;; Delete the version generated in the top directory, since we want them in the published notebooks or solutions
	(unless (member ipynb '("sitemap.ipynb" "theindex.ipynb"))
	  (delete-file ipynb))
	output))))


(defun ox-ipynb-publish-solution (plist filename pub-dir)
  "Publish an org-file to a Jupyter notebook."
  (let ((buf-open (find-buffer-visiting filename))
	(buf (find-file-noselect filename))
	(ipynb (file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))
    (with-current-buffer buf
      (let* ((output (ox-ipynb-export-to-ipynb-file)))
	(org-publish-attachment plist (expand-file-name output) pub-dir)
	(unless buf-open (kill-buffer buf))
	;; Delete the version generated in the top directory, since we want them in the published notebooks or solutions
	(unless (member ipynb '("sitemap.ipynb" "theindex.ipynb"))
	  (delete-file ipynb))
	output))))


(setq org-publish-project-alist
      '(("orgfiles"
         :base-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/"
         :base-extension "org"
         :publishing-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/notebooks"
	 :makeindex t
	 :auto-sitemap t
	 :sitemap-format-entry (lambda (fname style project)
				 (format "[[./%s][%s]]"
					 (concat (file-name-sans-extension fname) ".ipynb")
					 (concat (file-name-sans-extension fname) ".ipynb")))
         :publishing-function  ox-ipynb-publish-practice)

	("solutions"
	 :base-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/"
         :base-extension "org"
         :publishing-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/solutions"
	 ;; Change this... to making the functions
	 :publishing-function  ox-ipynb-publish-solution)

	("practice" :components ("orgfiles" "solutions"))))


(org-publish "practice" t)

(shell-command "git config advice.addIgnoredFile false")
(shell-command "git add *")
(shell-command "git commit -am \"published updates\"")
(shell-command "git push")